by: Jude Michael Teves

The CSV and XML portion are based from MSDS 2019 DMW class' Data Formats notebook.

## CSV

`C`omma-`s`eparated `V`alues

In [1]:
import pandas as pd
from io import StringIO
from pprint import pprint
from IPython.display import display_html

 - `io.StringIO` creates an in-memory file-like object
     - running `read()` on a file object places the pointer at the end of the file
     - `seek()` lets us move the pointer at any point (byte) in the file
 - `pprint` => pretty print, for Python arrays/dictionaries 
 - `display_html` => to "print" nicely formatted dataframes

### Pandas

In [2]:
pf1 = StringIO()
pf1.write('''1, 2, 3
4, 5, 6
7, 8, 9
''')

# '1,2,3\n4,5,6\n7,8,9\n'

24

To read the contents of the string buffer, we first have to set the pointer of the string to the first index.

In [3]:
pf1.seek(0) # sets the pointer to the first index.
pf1.read() # reads the string buffer

'1, 2, 3\n4, 5, 6\n7, 8, 9\n'

Pandas can read a string buffer as shown below

In [4]:
pf1.seek(0)
pd.read_csv(pf1, header=None)

0  1  2
0  1  2  3
1  4  5  6
2  7  8  9

We can also manually set where we would like to start the reading of the buffer. Note that this is the string in our buffer.
> '1, 2, 3\n4, 5, 6\n7, 8, 9\n'

If we start at index=7, then we would read the following string:
> '\n4, 5, 6\n7, 8, 9\n'

In [5]:
pf1.seek(7)
pd.read_csv(pf1, header=None)

0  1  2
0  4  5  6
1  7  8  9

We can also start at index=8
> '4, 5, 6\n7, 8, 9\n'

In [6]:
pf1.seek(8)
pd.read_csv(pf1, header=None)

0  1  2
0  4  5  6
1  7  8  9

What if the column sizes of each row is not consistent?

In [7]:
pf2 = StringIO()
pf2.write('''1, 2, 3
4, 5
7
''')
pf2.seek(0)
df_csv = pd.read_csv(pf2, header=None)
df_csv

0    1    2
0  1  2.0  3.0
1  4  5.0  NaN
2  7  NaN  NaN

Why are there `NaN`s in the dataframe?

There are `NaN`s because the column size of the dataframe is dictated by the first row.

We can also store the contents of a dataframe to a string buffer as shown below.

In [8]:
pf3 = StringIO()
df_csv.to_csv(pf3, index=False, header=False)
pf3.seek(0)
print(pf3.read())
pf3.seek(0)
pf3.read()

1,2.0,3.0
4,5.0,
7,,



'1,2.0,3.0\n4,5.0,\n7,,\n'

Why is the output different from the original csv?

When the string was put into a dataframe, some numbers turned into float and the null values were comma-separated.

---

If the column sizes are uneven, it will follow the size of the first column and will only read the rows that has equal or lower column size.

In [9]:
pf4 = StringIO()
pf4.write('''1
2, 3
4, 5, 6
''')
pf4.seek(0)
df_csv = pd.read_csv(pf4, header=None, error_bad_lines=False)
df_csv

b'Skipping line 2: expected 1 fields, saw 2\nSkipping line 3: expected 1 fields, saw 3\n'


0
0  1

In [10]:
pf4 = StringIO()
pf4.write('''1
2, 3
4, 5, 6
9
''')
pf4.seek(0)
df_csv = pd.read_csv(pf4, header=None, error_bad_lines=False)
df_csv

b'Skipping line 2: expected 1 fields, saw 2\nSkipping line 3: expected 1 fields, saw 3\n'


0
0  1
1  9

In [11]:
pf4 = StringIO()
pf4.write('''1,0
2, 3
4, 5, 6
9
''')
pf4.seek(0)
df_csv = pd.read_csv(pf4, header=None, error_bad_lines=False)
df_csv

b'Skipping line 3: expected 2 fields, saw 3\n'


0    1
0  1  0.0
1  2  3.0
2  9  NaN

### Writing CSV

In [12]:
import csv

In [13]:
a = [
    [1, 1.2, "a", None],
    ['''Multi-line
string''', 'string with "', 'string with ,'],
    [[1, 2], {'a': 1, 'b': 2}]
]

In [14]:
csv.list_dialects()

['excel', 'excel-tab', 'unix']

What's the difference between the CSV dialects?

- Unix just uses \n
- Excel(Windows) uses \r\n
- excel is comma-separated, excel-tab is... uhm. tab





In [15]:
cf1 = StringIO()
writer = csv.writer(cf1, dialect='excel')
writer.writerows(a)

In [16]:
cf1.seek(0)
csv_str = cf1.read()
print(repr(csv_str))
print('-----')
print(csv_str)

'1,1.2,a,\r\n"Multi-line\nstring","string with ""","string with ,"\r\n"[1, 2]","{\'a\': 1, \'b\': 2}"\r\n'
-----
1,1.2,a,
"Multi-line
string","string with ""","string with ,"
"[1, 2]","{'a': 1, 'b': 2}"



What do you notice about the CSV output?
- The first output is the string representation of the text. The newlines become \r\n(in Windows) or \n (in Unix).
- The second output is just the printed text in the csv.

In [17]:
cf1.seek(0)
reader = csv.reader(cf1)
csv_arr = list(reader)
pprint(csv_arr)

[['1', '1.2', 'a', ''],
 ['Multi-line\nstring', 'string with "', 'string with ,'],
 ['[1, 2]', "{'a': 1, 'b': 2}"]]


What's the difference of this output compared to the original CSV?

In here, the csv was turned into a list.

`CSV` doesn't have data types!

In [18]:
# Different number of columns per row

b = [[1],
    [2, 3],
    [4, 5, 6]
]
cf2 = StringIO()
writer = csv.writer(cf2)
writer.writerows(b)


cf2.seek(0)
reader = csv.reader(cf2)
csv_r = list(reader)
pprint(csv_r, width=20)

df_csv = pd.DataFrame(csv_r)
display_html(df_csv)
print(df_csv.dtypes)

[['1'],
 ['2', '3'],
 ['4', '5', '6']]


0 
 1 
 2 
 
 
 
 
 0 
 1 
 None 
 None 
 
 
 1 
 2 
 3 
 None 
 
 
 2 
 4 
 5 
 6

0    object
1    object
2    object
dtype: object


`csv.writer` and `csv.reader` works per row  
&nbsp;&nbsp;&rightarrow; different number of columns per row doesn't matter

## CSV Specification (RFC 4180)

   1.  Each record is located on a separate line, delimited by a line
       break
   2.  The last record in the file may or may not have an ending line
       break.
   3.  There maybe an optional header line appearing as the first line
       of the file with the same format as normal record lines.
   4.  Within the header and each record, 
       1. there may be one or more fields, separated by commas
       1. Each line should contain the same number of fields throughout the file
       1. Spaces are considered part of a field and should not be ignored
       1. The last field in the record must not be followed by a comma
   5.  Each field may or may not be enclosed in double quotes.
       If fields are not enclosed with double quotes, then
       double quotes may not appear inside the fields.
   6.  Fields containing line breaks, double quotes, and commas
       should be enclosed in double-quotes.
   7.  If double-quotes are used to enclose fields, then a double-quote
       appearing inside a field must be escaped by preceding it with
       another double quote.

<footer>https://tools.ietf.org/html/rfc4180</footer>

 ## XML

e`X`tensible `M`arkup `L`anguage

### Markup
 - **a system for marking or tagging a document that indicates its logical structure**
 - **derived from the traditional publishing practice of "marking up" a manuscript**

<footer>https://www.merriam-webster.com/dictionary/markup+language  
https://en.wikipedia.org/wiki/Markup_language
</footer>

Copy example XML from https://msdn.microsoft.com/en-us/library/ms762271(v=vs.85).aspx (also available in BlackBoard under DMW -> Data Formats)

```xml
<?xml version="1.0"?>
<catalog>
    <book id="bk101">
        <author>Gambardella, Matthew</author>
        <title>XML Developer's Guide</title>
        <genre>Computer</genre>
        <price>44.95</price>
        <publish_date>2000-10-01</publish_date>
        <description>An in-depth look at creating applications
            with XML.</description>
    </book>
    <book id="bk102">
        <author>Ralls, Kim</author>
        <title>Midnight Rain</title>
        <genre>Fantasy</genre>
        <price>5.95</price>
        <publish_date>2000-12-16</publish_date>
        <description>A former architect battles corporate zombies,
            an evil sorceress, and her own childhood to become queen
            of the world.</description>
    </book>
   ... 
</catalog>
```

or for a larger XML file, http://api.worldbank.org/v2/en/indicator/SP.POP.TOTL?downloadformat=xml
```xml
<?xml version="1.0" encoding="utf-8"?>
<Root xmlns:wb="http://www.worldbank.org">
  <data>
    <record>
      <field name="Country or Area" key="ABW">Aruba</field>
      <field name="Item" key="SP.POP.TOTL">Population, total</field>
      <field name="Year">1960</field>
      <field name="Value">54211</field>
    </record>
    <record>
      <field name="Country or Area" key="ABW">Aruba</field>
      <field name="Item" key="SP.POP.TOTL">Population, total</field>
      <field name="Year">1961</field>
      <field name="Value">55438</field>
    </record>
    ...
  </data>
</Root>
```



In [19]:
from xml.etree import ElementTree

In [20]:
tree = ElementTree.parse('catalog.xml')
root = tree.getroot()
root.tag

'catalog'

root is the topmost layer of the xml. We can use the following properties to check the details of any node:
- tag
- attrib
- text

To check the contents, we can get its children.

In [21]:
print('Tag name:', root.tag)
print('Attributes:', root.attrib)
print('Text:', root.text)

pprint(root.getchildren())

Tag name: catalog
Attributes: {}
Text: 
    
[<Element 'book' at 0x0000022AE2434688>,
 <Element 'book' at 0x0000022AE2468368>,
 <Element 'book' at 0x0000022AE24685E8>,
 <Element 'book' at 0x0000022AE2468818>,
 <Element 'book' at 0x0000022AE2468A98>,
 <Element 'book' at 0x0000022AE2468D18>,
 <Element 'book' at 0x0000022AE2468F48>,
 <Element 'book' at 0x0000022AE246A1D8>,
 <Element 'book' at 0x0000022AE246A408>,
 <Element 'book' at 0x0000022AE246A688>,
 <Element 'book' at 0x0000022AE246A8B8>,
 <Element 'book' at 0x0000022AE246AB88>]


We can do the same for the children nodes.

In [22]:
print('Tag name:', root[0].tag)
print('Attributes:', root[0].attrib)
print('Text:', root[0].text)

pprint(root[0].getchildren())

Tag name: book
Attributes: {'id': 'bk101'}
Text: 
        
[<Element 'author' at 0x0000022AE2434458>,
 <Element 'title' at 0x0000022AE24681D8>,
 <Element 'genre' at 0x0000022AE2468228>,
 <Element 'price' at 0x0000022AE2468278>,
 <Element 'publish_date' at 0x0000022AE24682C8>,
 <Element 'description' at 0x0000022AE2468318>]


As seen above, the first element of book is the author. To get all the authors of every book, we can do the following.

In [23]:
for i in root:
    pprint(i[0].text)

'Gambardella, Matthew'
'Ralls, Kim'
'Corets, Eva'
'Corets, Eva'
'Corets, Eva'
'Randall, Cynthia'
'Thurman, Paula'
'Knorr, Stefan'
'Kress, Peter'
"O'Brien, Tim"
"O'Brien, Tim"
'Galos, Mike'


We can do the same for title.

In [24]:
for i in root:
    pprint(i[1].text)

"XML Developer's Guide"
'Midnight Rain'
'Maeve Ascendant'
"Oberon's Legacy"
'The Sundered Grail'
'Lover Birds'
'Splish Splash'
'Creepy Crawlies'
'Paradox Lost'
'Microsoft .NET: The Programming Bible'
'MSXML3: A Comprehensive Guide'
'Visual Studio 7: A Comprehensive Guide'


### XPATH
Path expressions for navigating through a XML document

<table border="1">
  <colgroup>
    <col width="20%">
    <col width="70%">
  </colgroup>
  <thead valign="bottom">
  <tr><th>Syntax</th>
    <th>Meaning</th>
  </tr>
  </thead>
  <tbody valign="top">
  <tr><td><code><span>tag</span></code></td>
    <td>Selects all child elements with the given tag.
      For example, <code><span>spam</span></code> selects all child elements
      named <code><span>spam</span></code>, and <code><span>spam/egg</span></code> selects all
      grandchildren named <code><span>egg</span></code> in all children named
      <code><span>spam</span></code>.</td>
  </tr>
  <tr><td><code><span>\*</span></code></td>
    <td>Selects all child elements.  For example, <code><span>*/egg</span></code>
      selects all grandchildren named <code><span>egg</span></code>.</td>
  </tr>
  <tr><td><code><span>.</span></code></td>
    <td>Selects the current node.  This is mostly useful
      at the beginning of the path, to indicate that it’s
      a relative path.</td>
  </tr>
  <tr><td><code><span>//</span></code></td>
    <td>Selects all subelements, on all levels beneath the
      current  element.  For example, <code><span>.//egg</span></code> selects
      all <code><span>egg</span></code> elements in the entire tree.</td>
  </tr>
  <tr><td><code><span>..</span></code></td>
    <td>Selects the parent element.</td>
  </tr>
  <tr><td><code><span>[@attrib]</span></code></td>
    <td>Selects all elements that have the given attribute.</td>
  </tr>
  <tr><td><code><span>[@attrib='value']</span></code></td>
    <td>Selects all elements for which the given attribute
      has the given value.  The value cannot contain
      quotes.</td>
  </tr>
  <tr><td><code><span>[tag]</span></code></td>
    <td>Selects all elements that have a child named
      <code><span>tag</span></code>.  Only immediate children are supported.</td>
  </tr>
  <tr><td><code><span>[tag='text']</span></code></td>
    <td>Selects all elements that have a child named
      <code><span>tag</span></code> whose complete text content, including
      descendants, equals the given <code><span>text</span></code>.</td>
  </tr>
  <tr><td><code><span>[position]</span></code></td>
    <td>Selects all elements that are located at the given
      position.  The position can be either an integer
      (1 is the first position), the expression <code><span>last()</span></code>
      (for the last position), or a position relative to
      the last position (e.g. <code><span>last()-1</span></code>).</td>
  </tr>
  </tbody>
</table>

In [25]:
res = root.findall("book[@id='bk101']")
print(len(res))
print(ElementTree.tostring(res[0]).decode('utf8'))

1
<book id="bk101">
        <author>Gambardella, Matthew</author>
        <title>XML Developer's Guide</title>
        <genre>Computer</genre>
        <price>44.95</price>
        <publish_date>2000-10-01</publish_date>
        <description>An in-depth look at creating applications
            with XML.</description>
    </book>
    


To get all the titles with Eva Corets as author, we can use the following command.

In [26]:
res = root.findall("book[author='Corets, Eva']")
print(len(res))
for i in res:
    print(i[1].text)

3
Maeve Ascendant
Oberon's Legacy
The Sundered Grail


Read more, https://tools.ietf.org/html/rfc4825

## RegEx

![Regex](Python Regex Cheatsheet.png)

In [27]:
import re

**re.match**<br>

checks if the pattern matches the string. The pattern must occur at the start of the string. It returns the matched word. If the regex has a capturing group, denoted by `()`, then re.match will return the substrings that match the pattern inside the parenthesis `()`

In [28]:
text = 'I am Jude Jude Jude lala'
print(re.match(r'Jude', text))
print(re.match(r'I am', text).group(0))

s = re.match(r'(.*) am (.*)', text)
print(s.groups())
s = re.match(r'.*(Jude).*', text)
print(s.groups())


None
I am
('I', 'Jude Jude Jude lala')
('Jude',)


**Non-greedy searching**<br>

By default, + and * are greedy when it comes to searching. This means that it will try to capture everything until the last occurence of the pattern on the right. Here is an example.

In [29]:
s = re.match(r'(.*)Jude(.*)', text)
print(s.groups())

('I am Jude Jude ', ' lala')


In [30]:
print(len(s.groups()))
print(s[0]) # the first index contains the whole string
print(s[1]) # the second index contains the first captured group
print(s[2]) # the third index contains the second captured group

2
I am Jude Jude Jude lala
I am Jude Jude 
 lala


To prevent greedy searching, we add `?` after * or +.

In [31]:
s = re.match(r'(.*?)Jude(.*)', text)
print(s.groups())

('I am ', ' Jude Jude lala')


In [32]:
print(len(s.groups()))
print(s[0]) # the first index contains the whole string
print(s[1]) # the second index contains the first captured group
print(s[2]) # the third index contains the second captured group

2
I am Jude Jude Jude lala
I am 
 Jude Jude lala


**re.search**<br> 

similar to re.match but the pattern does not have to be at the start of the text

In [33]:
text = 'I am Jude Jude Jude lala'
s = re.search(r'Jude(.*)', text)

print(s.groups())

(' Jude Jude lala',)


**re.findall**

returns all strings that match the pattern.
By default, regex is multi-line. It means that the next line is not considered when searching a specific line. We can use the re.S flag to treat the text as 1 gigantic string.

In [34]:
logs = '''2018-07-12 10:53:32 - (network)[INFO][127.0.0.1:57534]: GET http://127.0.0.1:8081/v1/get/landuse?lng=121.02787971496582&lat=14.606591351724205&d=500  200 1883
2018-07-12 10:54:14 - (network)[INFO][127.0.0.1:57534]: GET http://127.0.0.1:8081/v1/get/landuse?lng=120.54113388061523&lat=14.67892177840028&d=500  200 1884'''

#date
s = re.findall(r'\d{4}-\d{2}-\d{2}', logs)
print(s)
#ip
s = re.findall(r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}', logs)
print(s)


['2018-07-12', '2018-07-12']
['127.0.0.1', '127.0.0.1', '127.0.0.1', '127.0.0.1']


Regex Lookahead is used to get words **before** a certain pattern

In [54]:
text = 'jude hello paul hello khalma hello meow hello lala hello michael hello end'

s = re.findall(r'.+(?=hello)', text)
print(s)

['jude hello paul hello khalma hello meow hello lala hello michael ']


To make the searching non-greedy, we add `?`

In [55]:
s = re.findall(r'.+?(?=hello)', text)
print(s)

['jude ', 'hello paul ', 'hello khalma ', 'hello meow ', 'hello lala ', 'hello michael ']


Get words after hello

In [88]:
s = re.findall(r'(?<=hello) \w+', text)
print(s)

[' paul', ' khalma', ' meow', ' lala', ' michael', ' end']


Get all words **between** hello

In [89]:
s = re.findall(r'(?<=hello).+?(?=hello)', text)
print(s)

[' paul ', ' khalma ', ' meow ', ' lala ', ' michael ']


Example using named captured group

In [90]:
text = '''abc 123 def 456 def 789
abc 123 def 456 def 789'''
s = re.findall(r'(?P<letters>\D{3}) (\d{3}) ((?P=letters))', text)
print(s)

[('def', '456', 'def'), ('def', '456', 'def')]


Answer for regex-logs homework.

In [91]:
logs = '''2018-07-12 10:53:32 - (network)[INFO][127.0.0.1:57534]: GET http://127.0.0.1:8081/v1/get/landuse?lng=121.02787971496582&lat=14.606591351724205&d=500  200 1883
2018-07-12 10:54:14 - (network)[INFO][127.0.0.1:57534]: GET http://127.0.0.1:8081/v1/get/landuse?lng=120.54113388061523&lat=14.67892177840028&d=500  200 1884
2018-07-12 10:54:17 - (network)[INFO][127.0.0.1:57534]: GET http://127.0.0.1:8081/v1/get/landuse?lng=120.27711868286133&lat=14.82400863570895&d=500  200 719
2018-07-12 10:54:20 - (network)[INFO][127.0.0.1:57534]: GET http://127.0.0.1:8081/v1/get/landuse?lng=120.91775894165039&lat=14.096787557861887&d=500  200 345
2018-07-12 11:48:05 - (sanic)[ERROR]: Traceback (most recent call last):
  File "/Users/edavid/anaconda3/envs/server/lib/python3.6/site-packages/sanic/router.py", line 356, in _get
    raise NotFound('Requested URL {} not found'.format(url))
sanic.exceptions.NotFound: Requested URL /v1/get/not_exist not found
2018-07-12 11:48:05 - (sanic)[ERROR]: Traceback (most recent call last):
  File "/Users/edavid/anaconda3/envs/server/lib/python3.6/site-packages/sanic/router.py", line 356, in _get
    raise NotFound('Requested URL {} not found'.format(url))
sanic.exceptions.NotFound: Requested URL /v1/get/not_exist not found
2018-07-12 10:54:20 - (network)[INFO][127.0.0.1:57534]: GET http://127.0.0.1:8081/v1/get/landuse?lng=120.91775894165039&lat=14.096787557861887&d=500  200 345
2018-07-12 11:48:05 - (sanic)[ERROR]: Traceback (most recent call last):
  File "/Users/edavid/anaconda3/envs/server/lib/python3.6/site-packages/sanic/router.py", line 356, in _get
    raise NotFound('Requested URL {} not found'.format(url))
sanic.exceptions.NotFound: Requested URL /v1/get/not_exist not found'''


lines = re.findall(r'.+?(?=\d{4}-\d{2})|.+', logs, flags=re.S)

log_list = []
for line in lines:
    date = re.search(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}', line).group(0)
    network = re.search(r'(?<=\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2} - \()\S+(?=\))', line).group(0)
    severity = re.search(r'(?<=\)\[)[A-Z]+(?=\])', line).group(0)
    
    content = {'date':date, 'network':network, 'severity':severity}
    if severity != 'ERROR':
        ip = re.search(r'\d{3}.\d{1}.\d{1}.\d{1}:\d{5}', line).group(0)
        method = re.search(r'\S+(?= http)', line).group(0)
        url = re.search(r'http\S+', line).group(0)
        response_code = re.search(r'(?<=  )\d+', line).group(0)
        bytes_ = re.search(r'(?<=  \d{3} )\d+', line).group(0)
        content['ip'] = ip
        content['method'] = method
        content['url'] = url
        content['response_code'] = response_code
        content['bytes'] = bytes_
        content['err_msg'] = None
    else:
        content['err_msg'] = re.search(r'(?<=\]: ).+', line, flags=re.S).group(0)
    log_list.append(content)
    
log_list


[{'bytes': '1883',
  'date': '2018-07-12 10:53:32',
  'err_msg': None,
  'ip': '127.0.0.1:57534',
  'method': 'GET',
  'network': 'network',
  'response_code': '200',
  'severity': 'INFO',
  'url': 'http://127.0.0.1:8081/v1/get/landuse?lng=121.02787971496582&lat=14.606591351724205&d=500'},
 {'bytes': '1884',
  'date': '2018-07-12 10:54:14',
  'err_msg': None,
  'ip': '127.0.0.1:57534',
  'method': 'GET',
  'network': 'network',
  'response_code': '200',
  'severity': 'INFO',
  'url': 'http://127.0.0.1:8081/v1/get/landuse?lng=120.54113388061523&lat=14.67892177840028&d=500'},
 {'bytes': '719',
  'date': '2018-07-12 10:54:17',
  'err_msg': None,
  'ip': '127.0.0.1:57534',
  'method': 'GET',
  'network': 'network',
  'response_code': '200',
  'severity': 'INFO',
  'url': 'http://127.0.0.1:8081/v1/get/landuse?lng=120.27711868286133&lat=14.82400863570895&d=500'},
 {'bytes': '345',
  'date': '2018-07-12 10:54:20',
  'err_msg': None,
  'ip': '127.0.0.1:57534',
  'method': 'GET',
  'network': '